In [8]:
# Part 1: Explore Dataset

from google.cloud import bigquery

client = bigquery.Client()

dataset_ref = client.dataset("openaq", project = "bigquery-public-data")
dataset = client.get_dataset(dataset_ref)

tables = list(client.list_tables(dataset))

for table in tables:
    print(table.table_id)

Using Kaggle's public dataset BigQuery integration.
global_air_quality


In [10]:
table_ref = dataset_ref.table("global_air_quality")

table = client.get_table(table_ref)

client.list_rows(table, max_results=5).to_dataframe()

,location,city,country,pollutant,value,timestamp,unit,source_name,latitude,longitude,averaged_over_in_hours,location_geom
0,"Borówiec, ul. Drapałka",Borówiec,PL,bc,0.85217,2022-04-28 07:00:00+00:00,µg/m³,GIOS,1.0,52.276794,17.074114,POINT(52.276794 1)
1,"Kraków, ul. Bulwarowa",Kraków,PL,bc,0.91284,2022-04-27 23:00:00+00:00,µg/m³,GIOS,1.0,50.069308,20.053492,POINT(50.069308 1)
2,"Płock, ul. Reja",Płock,PL,bc,1.41000,2022-03-30 04:00:00+00:00,µg/m³,GIOS,1.0,52.550938,19.709791,POINT(52.550938 1)
3,"Elbląg, ul. Bażyńskiego",Elbląg,PL,bc,0.33607,2022-05-03 13:00:00+00:00,µg/m³,GIOS,1.0,54.167847,19.410942,POINT(54.167847 1)
4,"Piastów, ul. Pułaskiego",Piastów,PL,bc,0.51000,2022-05-11 05:00:00+00:00,µg/m³,GIOS,1.0,52.191728,20.837489,POINT(52.191728 1)


In [19]:
# Part 2: Execute query
# 2.1 create Query
query = """
        SELECT city
        FROM `bigquery-public-data.openaq.global_air_quality`
        WHERE country = 'US'
        """


# 2.2 submit Query to dataset
query_job = client.query(query)

us_cities = query_job.to_dataframe()
us_cities.city.value_counts().head()

Using Kaggle's public dataset BigQuery integration.


/usr/local/lib/python3.11/dist-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


city
Phoenix-Mesa-Scottsdale                     39414
Los Angeles-Long Beach-Santa Ana            27479
Riverside-San Bernardino-Ontario            26887
New York-Northern New Jersey-Long Island    25417
San Francisco-Oakland-Fremont               22710
Name: count, dtype: int64

In [23]:
query2 = """
        SELECT city, country
        FROM `bigquery-public-data.openaq.global_air_quality`
        WHERE country = 'US'
"""

query_job2 = client.query(query2)

us_cities2 = query_job2.to_dataframe()
us_cities2.city.value_counts().head()



/usr/local/lib/python3.11/dist-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


city
Phoenix-Mesa-Scottsdale                     39414
Los Angeles-Long Beach-Santa Ana            27479
Riverside-San Bernardino-Ontario            26887
New York-Northern New Jersey-Long Island    25417
San Francisco-Oakland-Fremont               22710
Name: count, dtype: int64

In [27]:
us_cities2.country.value_counts()

country
US    1421351
Name: count, dtype: int64

In [30]:
query3 = """
        SELECT *
        FROM `bigquery-public-data.openaq.global_air_quality`
        WHERE country = 'US'
"""

query_job3 = client.query(query3)
us_aq = query_job3.to_dataframe()
us_aq.value_counts().head()

/usr/local/lib/python3.11/dist-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


location    city                    country  pollutant  value  timestamp                  unit   source_name  latitude  longitude  averaged_over_in_hours  location_geom     
1001        CA8 - ARB               US       pm25       3.300  2020-03-04 00:00:00+00:00  µg/m³  AirNow       1.0       38.684267  -120.98829              POINT(38.684267 1)    1
PORT HURON  Detroit-Warren-Livonia  US       o3         0.053  2022-05-10 16:00:00+00:00  ppm    AirNow       1.0       42.953300  -82.45640               POINT(42.9533 1)      1
                                                        0.055  2022-05-11 20:00:00+00:00  ppm    AirNow       1.0       42.953300  -82.45640               POINT(42.9533 1)      1
                                                        0.054  2022-05-15 22:00:00+00:00  ppm    AirNow       1.0       42.953300  -82.45640               POINT(42.9533 1)      1
                                                               2022-05-13 01:00:00+00:00  ppm    AirNow       

In [31]:
us_aq.columns.tolist()

['location',
 'city',
 'country',
 'pollutant',
 'value',
 'timestamp',
 'unit',
 'source_name',
 'latitude',
 'longitude',
 'averaged_over_in_hours',
 'location_geom']

In [32]:
us_aq.pollutant.value_counts().head()

pollutant
o3      509431
pm25    433374
pm10    163283
no2     123446
so2     104877
Name: count, dtype: int64

In [35]:
city_pollutant = us_aq.groupby(['city', 'pollutant']).size().reset_index(name='count')

print(city_pollutant)

           city pollutant  count
0           007      pm25    514
1           019      pm25      1
2           037      pm25    486
3           039      pm25      2
4           047      pm25    603
...         ...       ...    ...
2021  Yuba City      pm10    414
2022  Yuba City      pm25    429
2023       Yuma        o3    478
2024       Yuma      pm10    484
2025       Yuma      pm25    482

[2026 rows x 3 columns]
